In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
# date,county,state,fips,cases,deaths
foc = df.copy().drop(['fips','deaths'], axis=1).set_index(['date'])
foc = foc.loc[~((foc['county'] == 'Unknown') & (foc['state'] != 'Puerto Rico')),:]
#print(foc)
confirm = foc.groupby('state').sum()
#print(confirm.index)

date = pd.to_datetime("today")#.strftime('_%m_%d')
print('Latest update time is:',date)


Latest update time is: 2020-07-10 00:24:06.475533


In [3]:
abbrev = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}
in_abbrev = {v: k for k, v in abbrev.items()}
foc['abb'] = foc['state'].map(in_abbrev)

foc['co_st'] = foc['county']+', '+foc['abb']
collect = []
cols=['green days','last 14 days','County','State','both']
combined = foc.groupby('co_st').sum()
#print(combined)

#counter = 0
for j in combined.index:
    co = foc[foc['co_st'] == j]
    new = co['cases']-co['cases'].shift(1)
    window = 7
    ave = new#.rolling(window=window, min_periods=1).mean()
    if len(ave)<14:
        last_forteen = ave.sum()
    else:
        las = len(ave)-14
        last_forteen = ave[las:].sum()
    if last_forteen < 0:
        last_forteen = 0
    i = len(ave)-1
    c = 0
    while i > 0:
        if ave[i] <= 0:
            c = c + 1
        else:
            i = 0
        i = i - 1

    collect.append((c,
                   last_forteen,
                   foc.loc[foc.co_st == j,'county'][0],
                   foc.loc[foc.co_st == j,'state'][0],
                   j))
    #counter = counter +1
    #print(counter)

In [4]:
ze = pd.read_csv(r'zero.csv')
ze['all'] = ze['county'] + ', ' + ze['state']
ze['long'] = ze['state'].map(abbrev)
print(ze)


tody = pd.to_datetime("today").strftime('%y')
todm = pd.to_datetime("today").strftime('%m')
todd = pd.to_datetime("today").strftime('%d')

y=int(tody)+2000
m=int(todm)
d=int(todd)

from datetime import date
d0 = date(2020, 1, 20)
d1 = date(y,m,d)
delta = d1 - d0
delta = delta.days


for i in ze['all']:
    if i in combined.index:
        print('new')
        print(i)
    else:
        collect.append((int(delta),
                       0,
                       ze[ze['all'] == i]['county'].item(),
                       ze[ze['all'] == i]['long'].item(),
                       i))


           county state                all           long
0   Hoonah-Angoon    AK  Hoonah-Angoon, AK         Alaska
1         Skagway    AK        Skagway, AK         Alaska
2         Yakutat    AK        Yakutat, AK         Alaska
3         Dolores    CO        Dolores, CO       Colorado
4           Kiowa    CO          Kiowa, CO       Colorado
..            ...   ...                ...            ...
64      Stonewall    TX      Stonewall, TX          Texas
65        Daggett    UT        Daggett, UT           Utah
66          Wayne    UT          Wayne, UT           Utah
67       Garfield    WA       Garfield, WA     Washington
68      Doddridge    WV      Doddridge, WV  West Virginia

[69 rows x 4 columns]
new
Hickory, MO
new
Logan, ND
new
Butte, SD
new
Jones, SD


In [ ]:
##DIVISIONS Census Bureau

In [5]:
ne_data = {'New England':['Maine', 'New Hampshire', 'Vermont', 'Massachusetts', 'Rhode Island', 'Connecticut']}
ma_data = {'Middle Atlantic':['New York', 'New Jersey','Pennsylvania']}
en_data = {'East North Central':['Ohio', 'Michigan', 'Indiana', 'Wisconsin', 'Illinois']}
wn_data = {'West North Central':['Minnesota', 'Iowa', 'Missouri', 'North Dakota', 'South Dakota', 'Nebraska', 'Kansas']}
sa_data = {'South Atlantic':['Delaware', 'Maryland', 'West Virginia', 'Virginia', 'North Carolina', 'South Carolina', 'Georgia', 'Florida', 'Puerto Rico', 'District of Columbia']}
es_data = {'East South Central':['Kentucky', 'Tennessee', 'Alabama', 'Mississippi']}
ws_data = {'West South Central':['Arkansas', 'Louisiana', 'Oklahoma', 'Texas']}
mo_data = {'Mountain':['Montana', 'Idaho', 'Wyoming', 'Colorado', 'New Mexico', 'Arizona', 'Utah', 'Nevada']}
pa_data = {'Pacific':['California', 'Oregon', 'Washington', 'Alaska', 'Hawaii']}
ne = pd.DataFrame(ne_data)
ma = pd.DataFrame(ma_data)
en = pd.DataFrame(en_data)
wn = pd.DataFrame(wn_data)
sa = pd.DataFrame(sa_data)
es = pd.DataFrame(es_data)
ws = pd.DataFrame(ws_data)
mo = pd.DataFrame(mo_data)
pa = pd.DataFrame(pa_data)


div_l = [ne, ma, en, wn, sa, es, ws, mo, pa]
thr = pd.DataFrame(collect, columns=cols)
fin = thr.sort_values(['State','green days'], ascending=[True,False])
for k in fin['State']:
    for l in div_l:
        if k in l.values:
            fin.loc[(fin['State'] == k),'Division'] = l.columns[0]


fin['week'] = fin['green days'].gt(6) 
for d in div_l:
    d_name = d.columns[0]
    tab = fin[fin['Division']==d_name]
    tab = tab.sort_values(['week'], ascending=[False])
    tab_t = tab[tab['week']==True]
    tab_f = tab[tab['week']==False]
    tab_f = tab_f.sort_values(['last 14 days','green days'], ascending = [True,False])
    tab_t = tab_t.sort_values(['green days','last 14 days'], ascending = [False,True])
    tab = tab_t.append(tab_f)
    tab = tab.drop(['County','State','Division','week'], axis=1)
    tab2 = pd.DataFrame()
    tab2['County'] = tab['both']
    tab2['COVID-free days'] = tab['green days']
    tab2['Total cases in last 14 days'] = tab['last 14 days']
    tab2.reset_index(drop=True, inplace=True)
    tab2.index += 1
    tab2.to_excel (r'C:\Users\vince\Desktop\/%s.xlsx'% (d_name), index = True, header=True)